### Classificação de textos para análise de sentimentos

Base de dados 

Istruções:
- O objetivo deste trabalho é criar um modelo binário de aprendizado de máquina para classificação de textos. 
Para isso, será utilizado a base de dados [IMDb](http://ai.stanford.edu/~amaas/data/sentiment/), que consiste de dados textuais de críticas positivas e negativas de filmes
- Uma vez treinado, o modelo deve ter uma função `predict` que recebe uma string como parâmetro e retorna o valor 1 ou 0, aonde 1 significa uma crítica positiva e 0 uma crítica negativa
- O pré-processamento pode ser desenvolvidado conforme desejar (ex.: remoção de stopwords, word embedding, one-hot encoding, char encoding)
- É preferível que seja empregado um modelo de recorrência (ex.: rnn, lstm, gru) para a etapa de classificação
- Documente o código (explique sucintamente o que cada função faz, insira comentários em trechos de código relevantes)
- **Atenção**: Uma vez treinado o modelo final, salve-o no diretório do seu projeto e crie uma célula ao final do notebook contendo uma função de leitura deste arquivo, juntamente com a execução da função `predict`

Sugestões:
- Explorar a base de dados nas células iniciais do notebook para ter um melhor entendimento do problema, distribuição dos dados, etc
- Após desenvolver a estrutura de classificação, é indicado fazer uma busca de hiperparâmetros e comparar os resultados obtidos em diferentes situações

Prazo de entrega:
- 26-06-2021 às 23:59hs GMT-3

Formato preferível de entrega:
- Postar no portal Ava da disciplina o link do projeto no github (ou anexar o projeto diretamente no portal Ava)

luann.porfirio@gmail.com

In [ ]:
!pip install torchtext
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

In [25]:
import random
import re
import spacy
import torch
import torch.nn as nn
from torchtext.legacy import data
from torchtext.legacy import datasets
from typing import List

#### Pré-processamento do dataset, tratando:
- Remoção das tags HTML;
- Remoção das pontuações e demais caracteres não alfa-numéricos;
- Lematização das palavras;
- Remoção das "stop words";
- Filtro para manter apenas adjectivos, advérbios, substantivos e verbos

In [26]:
nlp = spacy.load("en_core_web_sm")
pos_filter = ['ADJ', 'ADV', 'NOUN', 'VERB'] #adjective, adverb, noun, verd

In [27]:
def process_review(review: str) -> List[str]:
    review = re.sub("<[^>]*>", "",  review) #Remove HTML tags
    review = re.sub("[^0-9a-zA-Z ]+", "",  review) #Remove qualquer caracter não alfa-numérico, menos o espaço
    processed_words = nlp(review)
    return [word.lemma_.strip().lower() for word in processed_words if not word.is_stop and word.pos_ in pos_filter]

#### Carregamento do dataset
- Utilizando uma semente fixa para melhor reprodução dos resultados;
- Carregamento das reviews utilizando o método de pré-processamento acima definido;
- Leitura dos labels como valores numéricos;

In [28]:
SEED = 1234

torch.manual_seed(SEED)

TEXT = data.Field(tokenize = process_review,
                  include_lengths = True)
LABEL = data.LabelField(dtype = torch.float)

#### Divisão do dataset
- Divisão feita para treino, desenvolvimento e teste

In [29]:
train_data, test_data = datasets.IMDB.splits(TEXT, LABEL)
train_data, dev_data = train_data.split(random_state = random.seed(SEED))

In [30]:
print(f"Train size: {len(train_data)}")
print(f"Dev size: {len(dev_data)}")
print(f"Test size: {len(test_data)}")

Train size: 17500
Dev size: 7500
Test size: 25000


#### Criação do vocabulário para treinamento, utilizando vetores pré-treinados

In [32]:
MAX_VOCAB_SIZE = 25_000

TEXT.build_vocab(train_data, 
                 max_size = MAX_VOCAB_SIZE, 
                 vectors = "glove.6B.100d", 
                 unk_init = torch.Tensor.normal_)

LABEL.build_vocab(train_data)

#### Verificação das palavras mais comuns no vocabulário construído

In [33]:
print(TEXT.vocab.freqs.most_common(20))

[('movie', 34341), ('film', 31711), ('good', 13713), ('time', 10109), ('character', 9564), ('watch', 9235), ('bad', 8833), ('story', 8593), ('see', 8590), ('think', 7990), ('scene', 7162), ('s', 6853), ('look', 6645), ('great', 6617), ('know', 6456), ('people', 6367), ('go', 5924), ('play', 5828), ('way', 5813), ('come', 5707)]


#### Verificação dos valores atribuídos aos labels de avaliação positiva e negativa

In [34]:
print(LABEL.vocab.stoi)

defaultdict(None, {'neg': 0, 'pos': 1})


####  Definição do tamanho dos lotes de processamento e criação dos iteradores para treino, desevolvimento e teste 
- Nos testes realizados não houve memória dedicada suficiente na GPU para treinamento do modelo, por isso foi fixado para treinar em CPU

In [38]:
BATCH_SIZE = 64

device = torch.device('cpu')

train_iterator, dev_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, dev_data, test_data), 
    batch_size = BATCH_SIZE,
    sort_within_batch = True, 
    device = device)

#### Definição do modelo
- Modelo LSTM bidirecional, com dropout e duas camadas;
- Criação de word embedding para melhor representação das palavras das revisões;
- Criada camada totalmente conectada de neurônios para receber as saída do LSTM e gerar uma saída única (que será utilizada para fazer a predição de positivo ou negativo);

In [39]:
class IMDB_LSTM(nn.Module):
    
    def __init__(self, vocab_size, pad_idx, embedding_dim=100, n_hidden=256, n_layers=2,
                               drop_prob=0.5, bidirectional=False):
        super().__init__()
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim, padding_idx = pad_idx)
        self.lstm = nn.LSTM(input_size=embedding_dim, 
                            hidden_size=n_hidden,  
                            num_layers=n_layers, 
                            dropout=drop_prob,
                            bidirectional=bidirectional)
        self.dropout = nn.Dropout(drop_prob)
        self.fc = nn.Linear(n_hidden * 2, 1)
      
    
    def forward(self, x, x_lengths): 
        embedded = self.dropout(self.embedding(x))
        
        packed_embedded = nn.utils.rnn.pack_padded_sequence(embedded, x_lengths.to('cpu'))
        
        packed_output, (hidden, cell) = self.lstm(packed_embedded)
        
        hidden = self.dropout(torch.cat((hidden[-2,:,:], hidden[-1,:,:]), dim = 1))
        hidden = self.fc(hidden)
        
        return hidden

#### Definição dos hiperparâmetros

In [40]:
VOCAB_SIZE = len(TEXT.vocab)
EMBEDDING_DIM = 100
N_HIDDEN = 256
N_LAYERS = 2
DROP_PROB = 0.5
LEARNING_RATE = 0.0001
N_EPOCHS = 5
BIDIRECTIONAL = True
PAD_IDX = TEXT.vocab.stoi[TEXT.pad_token]

#### Criação do modelo
- Definição da função de custo e do otimizador;
- Inicialização dos pesos pré-treinados do word embedding, zerando os tokens unk e pad conforme dica enontrada online pois inicialmente eles são irrelevantes para determinação do sentimento;

In [41]:
model = IMDB_LSTM(VOCAB_SIZE,
                  PAD_IDX,
                  embedding_dim=EMBEDDING_DIM, 
                  n_hidden=N_HIDDEN, 
                  n_layers=N_LAYERS, 
                  drop_prob=DROP_PROB, 
                  bidirectional=BIDIRECTIONAL,
                 ).to(device)

criterion = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

UNK_IDX = TEXT.vocab.stoi[TEXT.unk_token]

model.embedding.weight.data.copy_(TEXT.vocab.vectors)
model.embedding.weight.data[UNK_IDX] = torch.zeros(EMBEDDING_DIM)
model.embedding.weight.data[PAD_IDX] = torch.zeros(EMBEDDING_DIM)

#### Função de treinamento

In [42]:
def train(model, iterator, optimizer, criterion):
    epoch_loss = 0
    epoch_correct = 0
    
    model.train()
    
    for batch in iterator:
        text, text_lengths = batch.text
        label = batch.label
        
        optimizer.zero_grad()
        
        predictions = model(text, text_lengths)
        
        loss = criterion(predictions.squeeze(1), label)
        
        epoch_correct += torch.sum(torch.eq(predictions.argmax(1), label))
        
        loss.backward()
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator), epoch_correct / len(iterator)

#### Função de validação para desenvolvimento e testes

In [43]:
def evaluate(model, iterator, criterion):
    epoch_correct = 0
    
    model.eval()
    
    with torch.no_grad():
    
        for batch in iterator:
            text, text_lengths = batch.text
            label = batch.label
            
            predictions = model(text, text_lengths)
            epoch_correct += torch.sum(torch.eq(predictions.argmax(1), label))
        
    return epoch_correct / len(iterator)

#### Treinamento da primeira época separadamente para avaliar condição inicial do modelo antes de seguir com o restante do treinamento

In [19]:
train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
dev_acc = evaluate(model, dev_iterator, criterion)

print(f"Epoch: {1:02}")
print(f"\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%")
print(f"\tDev Acc: {dev_acc:.2f}%")

Epoch: 01
	Train Loss: 0.594 | Train Acc: 32.15%
	Dev Acc: 31.27%


#### Treinamento das demais época para completa 5 época de treinamento ao todo
- Alguma coisa está impactando na não evolução da acurrácia do modelo apesar do erro diminuir com o passar do tempo, mas não houve tempo para continuar a avaliação do que estava acontecendo;

In [20]:
for epoch in range(N_EPOCHS-1):
    train_loss, train_acc = train(model, train_iterator, optimizer, criterion)
    dev_acc = evaluate(model, dev_iterator, criterion)
    
    print(f"Epoch: {epoch+2:02}")
    print(f"\tTrain Loss: {train_loss:.3f} | Train Acc: {train_acc:.2f}%")
    print(f"\tDev Acc: {dev_acc:.2f}%")

Epoch: 02
	Train Loss: 0.482 | Train Acc: 32.15%
	Dev Acc: 31.27%
Epoch: 03
	Train Loss: 0.451 | Train Acc: 32.15%
	Dev Acc: 31.27%
Epoch: 04
	Train Loss: 0.431 | Train Acc: 32.15%
	Dev Acc: 31.27%
Epoch: 05
	Train Loss: 0.413 | Train Acc: 32.15%
	Dev Acc: 31.27%


#### Teste final do modelo

In [54]:
test_acc = evaluate(model, test_iterator, criterion)
print(f'Test Acc: {test_acc:.2f}%')

Test Acc: 31.97%


#### Persistência do modelo treinado

In [21]:
torch.save(model.state_dict(), 'IMDB_LSTM_bidirectional.pt')

#### Leitura do modelo treinado (necessário executar o código até antes da sessão de treinamento)

In [47]:
model.load_state_dict(torch.load('IMDB_LSTM_bidirectional.pt'))

<All keys matched successfully>

#### Função de predição do sentimento para uma senteção
- Retorna zero para sentimento negativo;
- Retorna um para sentimento positivo;

In [51]:
def predict(model, sentence):
    model.eval()
    tokenized = [tok.text for tok in nlp.tokenizer(sentence)]
    indexed = [TEXT.vocab.stoi[t] for t in tokenized]
    length = [len(indexed)]
    tensor = torch.LongTensor(indexed).to(device)
    tensor = tensor.unsqueeze(1)
    tensor_length = torch.LongTensor(length)
    prediction = torch.sigmoid(model(tensor, tensor_length))
    return int(round(prediction.item(), 0))

In [52]:
predict(model, "This film is terrible")

0

In [53]:
predict(model, "This film is great")

1